# Homework 5 - Andrea Lazzari

Consumer Resource Model with biotic resources

# Consumer and Resource Models

### Biotic resources

We want to analyze a more general case, the *MacArthur's Consumer Resource Model* with biotic resources.
Having $m$ species and $p$ resources, the model is the following:
$$
\begin{cases}

\dfrac{dn_{\sigma}}{dt} =  n_{\sigma} \cdot \bigg( \sum_{i=1}^{p}  v_i \cdot \alpha_{\sigma_i} \cdot r(c_i) - \delta_{\sigma} \bigg)  \quad \quad \sigma = 1, \dots, m \\
\\
\dfrac{dc_{i}}{dt} =  s_i - \sum_{\sigma = 1}^{m} n_{\sigma} \cdot \alpha_{\sigma_i} \cdot r(c_i)  \quad \quad \quad i = 1, \dots, p

\end{cases}
$$

The first equation represents the Consumer dynamic:

* $n_{\sigma}$ is the number of individuals of the $\sigma$-th species
* $v_i$ is the efficiency of the conversion of resource $i$ into biomass of species $\sigma$
* $\alpha_{\sigma_i}$ is the assimilation efficiency of the $\sigma$-th species, is also called **metabolic strategy** of $\sigma$
   - if $\alpha_{\sigma_i} > 0$ the species $\sigma$ is a consumer of resource $i$
* $\delta_{\sigma}$ is the death rate of the $\sigma$-th species


---

The second equation is instead representative of the Resources dynamic:

* $s_i$ is called supply rate and it is the maximum growth rate of the $i$-th resource
* $c_i$ is the concentration of the $i$-th resource
* $r_i(c_i)$ is the resource consumption rate of the $\sigma$-th species given by the *Monod function*:

$$r_i(c_i) = \dfrac{c_i}{c_i + K_i}$$

where $K_i$ is the half saturation constant of the $\sigma$-th species.

---

We study the case where resources are biotic and the  supply rate is given by the following expression:

$$ s(c_i) = \omega \cdot  c_i \cdot (1 - \frac{c_i}{K_i} ) $$

In addition, we assume to have a linear resource concentration $r(c_i) = c_i$.

## Simulation

In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd 
import plotly.express as px

from scipy.integrate import odeint


As far as the parameters are concerned, we draw them randomly from different distributions. In particular: 
* $v_i$ and $\delta_{\sigma}$ are drawn from a half normal. 
* The elelements of $\alpha_{\sigma_i}$ are drawn from a normal distribution putting the negative entries to 0.
* The $K_i$ are drawn from a uniform distribution
* $\omega$ is fixed equal to 1

In [7]:
#case m > p 

np.random.seed(2045247)

m = 5     #number of species
p = 3     #number of resources

n0 = list(np.abs(stats.norm.rvs(loc = 7, scale = 1, size = m)))
c0 = list(np.abs(stats.norm.rvs(loc = 7, scale = 1, size = p)))

alpha = np.random.normal(0, 1, size = (m, p))     #alpha_i
alpha[alpha < 0] = 0

v = np.random.normal(0, 1, size = p)        #v_i
v = np.abs(v)

delta = np.random.normal(0, 1, size = m)        #delta
delta = np.abs(delta)

k_i = np.random.uniform(0.5, 1.5, size = p)       #k_i


omega = 1



def system(y, t, m, p, alpha, v, k_i, delta, omega):
    return_list = []
    n_vec = np.array(y[:m])
    c_vec = np.array(y[m:])
    
    for i in range(m):
        n = y[i]
        dn_dt = n * ( np.sum(v * alpha[i, :] * c_vec) - delta[i])
        return_list.append(dn_dt)

    for j in range(p):
        
        c = y[m + j]
        s_i = omega * c * (1 - c/k_i[j])
        dc_dt = s_i - (np.sum(n_vec * alpha[:, j] * c))
        return_list.append(dc_dt)
    
    return return_list

t = np.linspace(0, 100, 300)   # time_domain

y0 = n0 + c0   # initial conditions

solution = odeint(system, y0, t, args=(m, p, alpha, v, k_i, delta, omega))


In [12]:
arr = np.vstack((t, np.array(solution).T)).T

df = pd.DataFrame(arr, columns = ['t[s]', 'Consumer_n1', 'Consumer_n2', 'Consumer_n3', 'Consumer_n4','Consumer_n5',
                                  'Resource_c1', 'Resource_c2', 'Resource_c3']) 



fig = px.line(df , x='t[s]', title='Consumer Resource Model - Biotic - with 5 species and 3', 
              y=['t[s]', 'Consumer_n1', 'Consumer_n2', 'Consumer_n3', 'Consumer_n4','Consumer_n5',
                    'Resource_c1', 'Resource_c2', 'Resource_c3'],
              labels={'value':'Dynamics', 'variable':'', 't[s]':'t [s]'},
              height=600, width=1200)

for i in range (1,6):
      fig.update_traces(selector = {'name': f'Consumer_n{i}'}, line = {'width':3.5})

for i in range (1,4):
      fig.update_traces(selector = {'name': f'Resource_c{i}'}, line = {'width':4, 'dash':'dash'})

fig.update_layout(font=dict(size=20), xaxis_range=[-5,104] , yaxis_range=[-1, 18])
fig.show()

As expected from the Competitive Exclusion Principle, in the case where $m > p $ the system is unstable and the number of species that survives is less or equal than $p$.

In [4]:
#case m < p 

np.random.seed(2045247)

m = 3     #number of species
p = 5     #number of resources

n0 = list(np.abs(stats.norm.rvs(loc = 7, scale = 1, size = m)))
c0 = list(np.abs(stats.norm.rvs(loc = 7, scale = 1, size = p)))

alpha = np.random.normal(0, 1, size = (m, p))     #alpha_i
alpha[alpha < 0] = 0

v = np.random.normal(0, 1, size = p)        #v_i
v = np.abs(v)

delta = np.random.normal(0, 1, size = m)        #delta
delta = np.abs(delta)

k_i = np.random.uniform(0.5, 1.5, size = p)       #k_i


omega = 1



def system(y, t, m, p, alpha, v, k_i, delta, omega):
    return_list = []
    n_vec = np.array(y[:m])
    c_vec = np.array(y[m:])
    
    for i in range(m):
        n = y[i]
        dn_dt = n * ( np.sum(v * alpha[i, :] * c_vec) - delta[i])
        return_list.append(dn_dt)

    for j in range(p):
        
        c = y[m + j]
        s_i = omega * c * (1 - c/k_i[j])
        dc_dt = s_i - (np.sum(n_vec * alpha[:, j] * c))
        return_list.append(dc_dt)
    
    return return_list

t = np.linspace(0, 100, 300)   # time_domain

y0 = n0 + c0   # initial conditions

solution = odeint(system, y0, t, args=(m, p, alpha, v, k_i, delta, omega))


In [6]:
arr = np.vstack((t, np.array(solution).T)).T

df = pd.DataFrame(arr, columns = ['t[s]', 'Consumer_n1', 'Consumer_n2', 'Consumer_n3', 'Resource_c1','Resource_c2',
                                  'Resource_c3', 'Resource_c4', 'Resource_c5']) 



fig = px.line(df , x='t[s]', title='Consumer Resource Model - Biotic - with 3 species and 5', 
              y=['t[s]', 'Consumer_n1', 'Consumer_n2', 'Consumer_n3', 'Resource_c1','Resource_c2',
                    'Resource_c3', 'Resource_c4', 'Resource_c5'],
              labels={'value':'Dynamics', 'variable':'', 't[s]':'t [s]'},
              height=600, width=1200)

for i in range (1,4):
      fig.update_traces(selector = {'name': f'Consumer_n{i}'}, line = {'width':3.5})

for i in range (1,6):
      fig.update_traces(selector = {'name': f'Resource_c{i}'}, line = {'width':4, 'dash':'dash'})

fig.update_layout(font=dict(size=20), xaxis_range=[-5,104] )
fig.show()

This time, in the case where $m < p$ in principle the number of species that survives can be equal to $m$.